In [17]:
from Utilities.CircuitEvaluation import get_quantum_engine
from projectq.ops import All, Measure
n_qubit=4
compiler_engine = get_quantum_engine()
wavefunction = compiler_engine.allocate_qureg(n_qubit) # Initialize the wavefunction

from Blocks import RotationEntangler

print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
entangler=RotationEntangler((1,2,3),(3,2,1))
print("The entangler to apply:")
print(entangler)
entangler.apply([0.5],wavefunction)
compiler_engine.flush()
print("The amplitude of |0000>:",compiler_engine.backend.get_amplitude([0,0,0,0], wavefunction))
entangler.is



The amplitude of |0000>: (1+0j)
The entangler to apply:
Type:RotationEntangler; Para Num:1; Qsubset:(1, 2, 3); Pauli:ZYX
The amplitude of |0000>: (0.8775825618903725+0j)


In [ ]:
All(Measure) | wavefunction
compiler_engine.flush()